In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot
# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()

from preprocessing import *
from mymodels import *
from databunch import *

data_path = Path('tbmData/data')
fn_txt = sorted(data_path.glob('*.txt'))
print(fn_txt[:3])
fn_cycles = Path('tbmData/cycles1')

# debug = True
debug = False

mulr = 50 if debug else 7

selected_columns = False
# selected_columns = True
if selected_columns == True:
    cont_names = ['推进速度', '主驱动1#电机扭矩', '刀盘功率', '刀盘扭矩', '刀盘转速','主液压油箱温度', '前点偏差X', '总推进力']
    n_cont = len(cont_names)
else:
    n_cont = 5 if debug else 192

# num_cycles = 5 if debug else 3481
num_cycles = 5 if debug else 500

valid_ratio = 0.2
train_ratio = 1 - valid_ratio
train_idx = np.arange(int(num_cycles * valid_ratio), num_cycles)
valid_idx = np.arange(int(num_cycles * valid_ratio))
train_idx_tile = (train_idx[:, None] + np.arange(mulr) * num_cycles).flatten()
valid_idx_tile = (valid_idx[:, None] + np.arange(mulr) * num_cycles ).flatten()  # take from all tiles

bs = int(num_cycles * train_ratio)
sl = 30
gpu_start = 2
torch.cuda.set_device(gpu_start)
# device_ids = range(gpu_start, gpu_start + num_gpus)

is_problem1 = True
# dep_var = ['桩号', '刀盘转速电位器设定值']    
dep_var = ['推进速度电位器设定值', '刀盘转速电位器设定值']

In [ ]:
postpond = 50
fns_feathers = sorted(fn_cycles.glob('cycle*'))[:num_cycles]
fmtr = DataFormatter(cycle_feathers=fns_feathers)
cycles = fmtr.cycles
idx = [beginning_index(o.iloc[:500]) + postpond for o in tqdm_notebook(cycles)]

In [ ]:
# cont_names = [o for o in cycles[0].columns[2:2+n_cont] if o not in dep_var]
cont_names = cont_names if selected_columns else [o for o in cycles[0].columns[2:] if o not in dep_var]
df_conts = tile_with_noise(cycles, idx, mulr, cont_names)

In [ ]:
deps = fmtr.get_y(dep_var)
deps = pd.concat([deps]*mulr).reset_index(drop=True)
cyc_cont = flatten_and_cat(df_conts, deps)

In [ ]:
cyc_cont.to_feather('tmp/cyc_cont_all_6_debug')

In [ ]:
#init
cyc_cont = feather.read_dataframe('tmp/cyc_cont_all_6_debug')

In [ ]:
#init
cyc_cont = feather.read_dataframe('tmp/cyc_cont_all_allc')

In [ ]:
valid_idx_tile = (valid_idx[:,None] + np.arange(mulr) * num_cycles).flatten() # take from all tiles
# procs = []
procs = [Normalize]
databunch = MultiDeptTabularDataBunch.from_df('tmp', cyc_cont, dep_var, valid_idx=valid_idx_tile, bs=bs, procs=procs)

rnndb = RNNDataBunch.create(databunch.train_ds, databunch.valid_ds, bs=bs)

In [ ]:
n_hidden, n_layers = n_cont * 3, 2
# hidden_p, input_p, embed_p, weight_p = 0,0,0,0
hidden_p, input_p, weight_p = 0.3, 0.6, 0.5

rnn_enc = ContModel1(1, n_cont, n_hidden, n_layers, 
                     hidden_p=hidden_p, input_p=input_p, embed_p=0, weight_p=weight_p)

layers = [3*n_cont, 5, 2]
# drops = [0, 0, 0, 0]
drops = [0.1] * 4
# model = MultiInputSequentialRNN(rnn_enc, DummyModel(layers, drops)).cuda()
# model = DummyContModel1(1, n_cont, n_hidden, n_layers)
model = MultiInputSequentialRNN(rnn_enc, PoolingLinearClassifier(layers, drops)).cuda()
model.reset()
    
weight = cyc_cont[dep_var].max().values
weight = torch.tensor(weight[::-1] / weight.max(), dtype=torch.float32).cuda()

learner = Learner(rnndb, model, loss_func=weighted_rnn_mse(weight), metrics=rnn_metrics, opt_func=optim.SGD)
# learner = Learner(rnndb, model, loss_func=weighted_our_loss(weight), metrics=rnn_metrics, opt_func=optim.SGD)

from fastai.callbacks import *
# learner.callback_fns += [ShowGraph, partial(SaveModelCallback, name='rnn0')]
learner.callback_fns += [ShowGraph,]
learner.callbacks += [TerminateOnNaNCallback()]
alpha, beta = 2., 1.
# alpha, beta = 0., 0.
learner.callbacks.append(RNNTrainer(learner, sl, alpha=alpha, beta=beta))

clip = 1e0
learner.callback_fns.append(partial(GradientClipping, clip=clip))
split_func = lambda x: list(x)
learner.split(split_func)

In [ ]:
learner.freeze_to(1)
learner.fit(40, [1e-1, 1])

In [ ]:
learner.unfreeze()
learner.lr_find(1e-10, num_it=240)
learner.recorder.plot(1,1)

In [ ]:
lr = 1e-1 * 3
learner.fit_one_cycle(10, [lr/3, lr])

In [ ]:
lr = 1e-1
learner.fit(4, lr)

In [ ]:
learner.callback_fns[-1].keywords['clip'] = .1
learner.callback_fns[-1].keywords

In [ ]:
learner.fit(20, 1e-1)

In [ ]:
learner.fit(30, 1e-2)

In [ ]:
# x, y = ni(learner.data.train_dl)
x, y = ni(learner.data.valid_dl)
y, p = y.cpu(), learner.model(*x)[0].cpu()
y_np, p_np = to_np(y), to_np(p)
# our_metrics(y, p), our_metrics_np(y_np, p_np)
lf = weighted_rnn_mse(weight.cpu())
our_metrics_np(p_np, y_np)

In [ ]:
i = 0
list(zip(p_np[:,i], y_np[:,i]))

In [ ]:
p = learner.model(*x)[1]

In [ ]:
# l x b x t x h
l, b = 0, 0
[(float(p[l][b][t].mean()), float(p[l][b][t].std())) for t in range(30)]

In [ ]:
# l x b x t x h
l, b, t = 0, 0, 1
[(float(p[l][b][t].mean()), float(p[l][b][t].std())) for l in range(n_layers)]

In [ ]:
scatter = 

In [ ]:
list(zip(y_np.tolist(), p_np.tolist()))